<a href="https://colab.research.google.com/github/yanamushka/ML-course/blob/main/%22HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [140]:
raw_df = pd.read_csv('/content/customer_segmentation_train.csv')

In [141]:
raw_df.head(5)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [278]:
raw_df.Segmentation.value_counts()

,count
Segmentation,
D,2268
A,1972
C,1970
B,1858


In [142]:
raw_df.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [143]:
null_count = raw_df.isnull().sum()

In [144]:
null_precent = null_count/raw_df.shape[0]

In [145]:
null_df = pd.DataFrame({'column':raw_df.columns,'null_count': null_count, 'null_precent': null_precent, 'cat':raw_df.dtypes})

In [146]:
null_df.sort_values(by = 'null_precent', ascending = False)

,column,null_count,null_precent,cat
Work_Experience,Work_Experience,829,0.102752,float64
Family_Size,Family_Size,335,0.041522,float64
Ever_Married,Ever_Married,140,0.017353,object
Profession,Profession,124,0.015369,object
Graduated,Graduated,78,0.009668,object
Var_1,Var_1,76,0.009420,object
ID,ID,0,0.000000,int64
Gender,Gender,0,0.000000,object
Age,Age,0,0.000000,int64
Spending_Score,Spending_Score,0,0.000000,object


Для числових стовпців перевіримо медіану, моду і середнє значення і замінемо пропущені значення одним з параметрів. Для категоріальних колонок замінимо пропущені значення на 'unknown'

In [147]:
print(f'mean {raw_df.Work_Experience.mean()}')
print(f'mode {raw_df.Work_Experience.mode()}')
print(f'median {raw_df.Work_Experience.median()}')

mean 2.641663213150988
mode 0    1.0
Name: Work_Experience, dtype: float64
median 1.0


In [148]:
raw_df.Work_Experience.fillna(1, inplace = True)

<ipython-input-148-6c2ba0486afa>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  raw_df.Work_Experience.fillna(1, inplace = True)


In [149]:
print(f'mean {raw_df.Family_Size.mean()}')
print(f'mode {raw_df.Family_Size.mode()}')
print(f'median {raw_df.Family_Size.median()}')

mean 2.85012285012285
mode 0    2.0
Name: Family_Size, dtype: float64
median 3.0


In [150]:
raw_df.fillna({'Family_Size':2}, inplace = True)

In [151]:
raw_df.fillna({'Ever_Married':'Unknown'}, inplace = True)

In [152]:
raw_df.fillna({'Profession':'Unknown'}, inplace = True)
raw_df.fillna({'Graduated':'Unknown'}, inplace = True)
raw_df.fillna({'Var_1':'Unknown'}, inplace = True)

In [153]:
raw_df.isnull().sum()

,0
ID,0
Gender,0
Ever_Married,0
Age,0
Graduated,0
Profession,0
Work_Experience,0
Spending_Score,0
Family_Size,0
Var_1,0


In [154]:
prepared_df = raw_df.copy()

In [155]:
input_cols = list(prepared_df.columns)[1:-1]
target_col = 'Segmentation'

In [156]:
cat_cols = prepared_df[input_cols].select_dtypes(include = 'object').columns.tolist()
num_cols = prepared_df[input_cols].select_dtypes(exclude = 'object').columns.tolist()

In [157]:

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split


In [158]:
prepared_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,1.0,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,1.0,High,6.0,Cat_6,A


Колонки Gender, Ever_Married, Graduated, Spending_Score - закодуємо за допомогою LabelEncoder.

In [159]:
label_cols = ['Gender', 'Ever_Married', 'Graduated', 'Spending_Score']
one_hot_cols = ['Profession', 'Var_1']

In [160]:
lb = LabelEncoder()

In [161]:
for col in label_cols:
  prepared_df[col] = lb.fit_transform(prepared_df[col])


In [162]:
onehot = OneHotEncoder(sparse_output = False)

In [163]:
encoded_cols = onehot.fit_transform(prepared_df[one_hot_cols])

In [164]:
columns=onehot.get_feature_names_out(one_hot_cols)

In [165]:
prepared_df[columns] = encoded_cols

In [166]:
prepared_df.drop(['Profession', 'Var_1'], axis = 1, inplace = True)

In [167]:
scaler = MinMaxScaler()

In [168]:
train_df, test_df = train_test_split(prepared_df, test_size = 0.2, random_state = 42)

In [169]:
new_input = [col for col in prepared_df.columns if col not in [target_col]]

In [170]:
train_input, train_target = train_df[new_input], train_df[target_col]
test_input, test_target = test_df[new_input], test_df[target_col]

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [187]:
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

In [259]:
train_raw, test_raw = train_test_split(prepared_df, test_size = 0.2, random_state = 42)

In [173]:
y_target = 'Segmentation'

In [266]:
x_cols = [col for col in prepared_df.columns if col != y_target]


In [265]:
X = train_raw[x_cols]
y = train_raw[y_target]

In [176]:
cat_cols = X.select_dtypes(include=['object']).columns

In [196]:
cat_index = [X.columns.get_loc(col) for col in cat_cols]

In [197]:
cat_index

[1, 2, 4, 5, 7, 9]

In [198]:
smote_nc = SMOTENC( categorical_features=cat_index, random_state=42)

In [199]:
X_resampled, y_resampled = smote_nc.fit_resample(X, y)

In [272]:
for col in label_cols:
  X_resampled[col] = lb.fit_transform(X_resampled[col])

In [273]:
res_encod = onehot.transform(X_resampled[['Profession', 'Var_1']])

In [274]:
X_res_cod_cols = onehot.get_feature_names_out(['Profession', 'Var_1'])

In [275]:
X_resampled[X_res_cod_cols] = res_encod

In [276]:
X_resampled.drop(['Profession', 'Var_1'], axis = 1, inplace = True)

In [213]:
for col in label_cols:
  X[col] = lb.fit_transform(X[col])

<ipython-input-213-17aba012dba8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = lb.fit_transform(X[col])
<ipython-input-213-17aba012dba8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = lb.fit_transform(X[col])
<ipython-input-213-17aba012dba8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [239]:
encoded_x = onehot.transform(X[['Profession', 'Var_1']])

In [240]:
X_coded_cols = onehot.get_feature_names_out(['Profession', 'Var_1'])

In [241]:
X[X_coded_cols] = encoded_x

In [243]:
X.drop(['Profession', 'Var_1'], axis = 1, inplace = True)

In [269]:
smotenc_resample = SMOTENC(categorical_features=cat_index, random_state=42)

In [270]:
smotetomek = SMOTETomek(smote = smotenc_resample, random_state = 0)

In [246]:
x_smote_tomek, y_smote_tomek = smotetomek.fit_resample(X, y)

In [247]:
x_smote_tomek

,ID,Gender,Ever_Married,Age,Graduated,Work_Experience,Spending_Score,Family_Size,Profession_Artist,Profession_Doctor,...,Profession_Marketing,Profession_Unknown,Var_1_Cat_1,Var_1_Cat_2,Var_1_Cat_3,Var_1_Cat_4,Var_1_Cat_5,Var_1_Cat_6,Var_1_Cat_7,Var_1_Unknown
0,460459,0,0,33,0,0.0,2,5.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,0.0
1,459043,0,2,49,0,1.0,2,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.0,0.0
2,461220,0,0,26,0,8.0,2,3.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0
3,460403,0,0,56,2,3.0,2,1.0,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0
4,462641,0,2,42,0,1.0,0,2.0,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5113,461787,1,2,80,2,1.0,1,2.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0
5114,464550,1,2,46,2,1.0,1,4.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.273909,0.0,0.726091,0.0,0.0
5115,467228,0,0,29,0,1.0,2,4.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0
5116,466640,1,2,47,2,6.0,0,2.0,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [249]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

In [255]:
def model_and_report(input, target, test, name=''):
  log_reg = LogisticRegression(solver = 'liblinear', random_state = 42)
  ovr_model = OneVsRestClassifier(log_reg)
  ovr_model.fit(input, target)
  ovr_pred = ovr_model.predict(test)
  print(f'Classification report for {name}')
  print(classification_report(test_target, ovr_pred))

In [256]:
model_and_report(train_input, train_target, test_input, 'original')

Classification report for original
              precision    recall  f1-score   support

           A       0.00      0.00      0.00       391
           B       0.00      0.00      0.00       369
           C       0.00      0.00      0.00       380
           D       0.29      1.00      0.45       474

    accuracy                           0.29      1614
   macro avg       0.07      0.25      0.11      1614
weighted avg       0.09      0.29      0.13      1614



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [277]:
model_and_report(X_resampled, y_resampled, test_input, 'smote')

Classification report for smote
              precision    recall  f1-score   support

           A       0.00      0.00      0.00       391
           B       0.00      0.00      0.00       369
           C       0.24      1.00      0.38       380
           D       0.00      0.00      0.00       474

    accuracy                           0.24      1614
   macro avg       0.06      0.25      0.10      1614
weighted avg       0.06      0.24      0.09      1614



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [268]:
model_and_report(x_smote_tomek, y_smote_tomek, test_input, 'smote_tomek')

Classification report for smote_tomek
              precision    recall  f1-score   support

           A       0.00      0.00      0.00       391
           B       0.00      0.00      0.00       369
           C       0.34      0.74      0.46       380
           D       0.47      0.78      0.59       474

    accuracy                           0.40      1614
   macro avg       0.20      0.38      0.26      1614
weighted avg       0.22      0.40      0.28      1614



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Для визначальнох метрики обираю показник macro average так як маємо більш менш збалансовані класи і потрібно оцінити як модель передбачає кожен з них

Найкращий результат має модель з використанням Smote Tomek, так як вона має найкращий показник macro average 0.26 проти 0.10 та 0.11 у інших моделей.